In [1]:
from ipyparallel import Client
import pandas
import pymongo

In [2]:
client = pymongo.MongoClient()
db = client.swatford

In [3]:
descs = pandas.DataFrame([doc for doc in db.medline.descs.find({},{"_id":0})])

In [4]:
descs = descs[["uid","ancestors","tn"]].copy()

In [6]:
mapped_descs = set([doc["_id"] for doc in db.medline.descs.mappings.aggregate([{"$group":{"_id":"$uid"}}])],
                   allowDiskUse=True)

In [10]:
descs = descs[descs.uid.isin(mapped_descs)].copy()

In [11]:
tn_data = []
for row in descs[descs.tn.isnull()==False].itertuples():
    for tn in row.tn:
        tn_data.append({"uid":row.uid,"tn":tn})

In [12]:
an_data = []
for row in descs[descs.tn.isnull()==False].itertuples():
    for an in row.ancestors:
        an_data.append({"child":row.uid,"an_tn":an})

In [13]:
an_map = pandas.DataFrame(an_data).merge(pandas.DataFrame(tn_data),left_on="an_tn",right_on="tn")[["child","uid"]]

In [14]:
an_map.columns = ["uid","ancestor"]

In [15]:
pmids = [doc["_id"] for doc in  db.medline.descs.mappings.aggregate([{"$group": {"_id":"$pmid"}}],
                                                                         allowDiskUse=True)]

In [16]:
rc = Client()

In [17]:
with rc[:].sync_imports():
    import pymongo

importing pymongo on engine(s)


In [20]:
for i in rc.ids:
    rc[i].push({"pmids":pmids[(len(pmids)//len(rc.ids)+1)*i:(len(pmids)//len(rc.ids)+1)*(i+1)].copy(),"an_map":an_map})

In [21]:
%%px
client = pymongo.MongoClient()
db = client.swatford

In [22]:
def desc_ancestor_map():
    to_insert = []
    for i,pmid in enumerate(pmids):
        uids = db.medline.descs.mappings.distinct("uid",{"pmid":pmid})
        ans = list(an_map[an_map.uid.isin(uids)].ancestor.values)
        to_insert+=[{"pmid":pmid,"uid":uid} for uid in set(uids+ans)]
        if i%10000==0:
            db.medline.descs.ancestors.mappings.insert_many(to_insert)
            to_insert = []
            print(i)
    db.medline.descs.ancestors.mappings.insert_many(to_insert)

In [23]:
ar = rc[:].apply_async(desc_ancestor_map)

In [32]:
ssi_list = pandas.read_pickle("../data/SSI/ssi_gene_list.p")

In [34]:
ssi_list.to_csv("../data/SSI/ssi_gene_list.csv",index=False)

In [40]:
total = 0
for x in ar.stdout:
    total += int(x.split("\n")[-2])
    print(x.split("\n")[-2])
print(f"Total: {total}, "+str(len(pmids)))

2950000
2950000
2950000
2950000
2950000
2950000
2950000
2950000
Total: 23600000, 23615049


In [41]:
ar.completed

[datetime.datetime(2017, 6, 29, 8, 6, 21, 117389, tzinfo=tzlocal()),
 datetime.datetime(2017, 6, 29, 7, 55, 52, 165858, tzinfo=tzlocal()),
 datetime.datetime(2017, 6, 29, 8, 1, 6, 719720, tzinfo=tzlocal()),
 datetime.datetime(2017, 6, 29, 8, 9, 34, 650542, tzinfo=tzlocal()),
 datetime.datetime(2017, 6, 29, 7, 30, 36, 747849, tzinfo=tzlocal()),
 datetime.datetime(2017, 6, 29, 8, 4, 37, 505891, tzinfo=tzlocal()),
 datetime.datetime(2017, 6, 29, 8, 10, 23, 142609, tzinfo=tzlocal()),
 datetime.datetime(2017, 6, 29, 8, 10, 30, 627049, tzinfo=tzlocal())]